In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from keras.layers import Dense
from keras.models import Sequential
import string
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# reading data

In [2]:
CSV_COLUMN_NAMES = ['description','label']
data = pd.read_csv('data.csv',names=CSV_COLUMN_NAMES, header=0)
data.head()

,description,label
0,"♦️ I wish you to stay healthy. Your character,...",0
1,Les bases de l'algorithmique b Darija vos rema...,0
2,Corona restrictions causing terrible financial...,0
3,📣📣AMAZON recrute des conseillers clients à Dom...,1
4,Great illustration....,0


# text management

In [3]:
Stopwords = {''}
Stopwords_fr=set(stopwords.words('french'))
Stopwords_eng = set(stopwords.words('english'))
add_stop_words = "all almost also any because cause could do does done don't end for never see soon sorry than ourselves hers between yourself ut again there about once during out very having with they own an be some for do its yours such into of most itself other off is s am for who as from him each the themselves until below are we these your his through don nor me were her more himself this down should our their while above both up to ours had she all no when at any before them same and been have in will on does yourselves then that because what over why so can did not now under he you herself has just where too only myself which those i after few whom t being if theirs my against a by doing it how further was here than a à â abord afin ah ai aie ainsi allaient allo allô allons après assez attendu au aucun aucune aujourd aujourd'hui auquel aura auront aussi autre autres aux auxquelles auxquels avaient avais avait avant avec avoir ayant b bah beaucoup bien bigre boum bravo brrr c ça car ce ceci cela celle celle-ci celle-là celles celles-ci celles-là celui celui-ci celui-là cent cependant certain certaine certaines certains certes ces cet cette ceux ceux-ci ceux-là chacun chaque cher chère chères chers chez chiche chut ci cinq cinquantaine cinquante cinquantième cinquième clac clic combien comme comment compris concernant contre couic crac d da dans de debout dedans dehors delà depuis derrière des dès désormais desquelles desquels dessous dessus deux deuxième deuxièmement devant devers devra différent différente différentes différents dire divers diverse diverses dix dix-huit dixième dix-neuf dix-sept doit doivent donc dont douze douzième dring du duquel durant e effet eh elle elle-même elles elles-mêmes en encore entre envers environ es ès est et etant étaient étais était étant etc été etre être eu euh eux eux-mêmes excepté f façon fais faisaient faisant fait feront fi flac floc font g gens h ha hé hein hélas hem hep hi ho holà hop hormis hors hou houp hue hui huit huitième hum hurrah i il ils importe j je jusqu jusque k l la là laquelle las le lequel les lès lesquelles lesquels leur leurs longtemps lorsque lui lui-même m ma maint mais malgré me même mêmes merci mes mien mienne miennes miens mille mince moi moi-même moins mon moyennant n na ne néanmoins neuf neuvième ni nombreuses nombreux non nos notre nôtre nôtres nous nous-mêmes nul o o| ô oh ohé olé ollé on ont onze onzième ore ou où ouf ouias oust ouste outre p paf pan par parmi partant particulier particulière particulièrement pas passé pendant personne peu peut peuvent peux pff pfft pfut pif plein plouf plus plusieurs plutôt pouah pour pourquoi premier première premièrement près proche psitt puisque q qu quand quant quanta quant-à-soi quarante quatorze quatre quatre-vingt quatrième quatrièmement que quel quelconque quelle quelles quelque quelques quelqu'un quels qui quiconque quinze quoi quoique r revoici revoilà rien s sa sacrebleu sans sapristi sauf se seize selon sept septième sera seront ses si sien sienne siennes siens sinon six sixième soi soi-même soit soixante son sont sous stop suis suivant sur surtout t ta tac tant te té tel telle tellement telles tels tenant tes tic tien tienne tiennes tiens toc toi toi-même ton touchant toujours tous tout toute toutes treize trente très trois troisième troisièmement trop tsoin tsouin tu u un une unes uns v va vais vas vé vers via vif vifs vingt vivat vive vives vlan voici voilà vont vos votre vôtre vôtres vous vous-mêmes vu w x y z zut"
add_stop_words = add_stop_words.split()
Stopwords.update(add_stop_words)
Stopwords.update(Stopwords_fr)
Stopwords.update(Stopwords_eng)

def stemSentence(sentence):
    token_words=word_tokenize(str(sentence))
    token_words
    porter = PorterStemmer()
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

punctuations = string.punctuation #+"’¶•@°©®™"

def funct(text):
    text = str(text).lower()
    
    translator = str.maketrans(punctuations," "*len(punctuations))
    s = text.translate(translator)
    
    res = ''.join([i for i in s if not i.isdigit()])
    
    wordtokens = word_tokenize(res)
    
    return  ' '.join([w for w in wordtokens if not w in Stopwords])


In [4]:
data['description']= data['description'].apply(stemSentence)
data['description']=data['description'].apply(funct)

freq = pd.Series(' '.join(data['description']).split()).value_counts()[:3]
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

data.dropna(inplace=True)

In [5]:
vectorizer = TfidfVectorizer()

Y = data.pop('label')
X = vectorizer.fit_transform(data['description'])


In [6]:
X.shape[1]

8110

# training phase

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=60)

In [9]:
model = Sequential()
model.add(Dense(50, input_dim = X.shape[1] , activation='relu'))
model.add(Dense(70 , activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=15)

_, accuracy = model.evaluate(X_test, Y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/500
584/584 [==============================] - 1s 950us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 2/500
584/584 [==============================] - 0s 394us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 3/500
584/584 [==============================] - 0s 389us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 4/500
584/584 [==============================] - 0s 396us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 5/500
584/584 [==============================] - 0s 413us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 6/500
584/584 [==============================] - 0s 412us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 7/500
584/584 [==============================] - 0s 408us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 8/500
584/584 [==============================] - 0s 422us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 9/500
584/584 [==============================] - 0s 417us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 10/500
584/584 [==============================] - 0s 403us/step - l

584/584 [==============================] - 0s 454us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 80/500
584/584 [==============================] - 0s 456us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 81/500
584/584 [==============================] - 0s 453us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 82/500
584/584 [==============================] - 0s 451us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 83/500
584/584 [==============================] - 0s 439us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 84/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 85/500
584/584 [==============================] - 0s 446us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 86/500
584/584 [==============================] - 0s 456us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 87/500
584/584 [==============================] - 0s 475us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 88/500
584/584 [==============================] - 0s 439us/step - loss:

584/584 [==============================] - 0s 446us/step - loss: 8.4543 - accuracy: 0.44860s - loss: 8.7724 - accuracy: 0.
Epoch 158/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 159/500
584/584 [==============================] - 0s 442us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 160/500
584/584 [==============================] - 0s 434us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 161/500
584/584 [==============================] - 0s 437us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 162/500
584/584 [==============================] - 0s 444us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 163/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 164/500
584/584 [==============================] - 0s 434us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 165/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 166/500
584/584 [==============

584/584 [==============================] - 0s 447us/step - loss: 8.4543 - accuracy: 0.44860s - loss: 8.1826 - accuracy: 0.
Epoch 235/500
584/584 [==============================] - 0s 444us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 236/500
584/584 [==============================] - 0s 439us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 237/500
584/584 [==============================] - 0s 439us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 238/500
584/584 [==============================] - 0s 439us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 239/500
584/584 [==============================] - 0s 434us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 240/500
584/584 [==============================] - 0s 437us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 241/500
584/584 [==============================] - 0s 446us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 242/500
584/584 [==============================] - 0s 444us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 243/500
584/584 [==============

584/584 [==============================] - 0s 446us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 312/500
584/584 [==============================] - 0s 439us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 313/500
584/584 [==============================] - 0s 444us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 314/500
584/584 [==============================] - 0s 430us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 315/500
584/584 [==============================] - 0s 439us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 316/500
584/584 [==============================] - 0s 475us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 317/500
584/584 [==============================] - 0s 434us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 318/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 319/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 320/500
584/584 [==============================] - 0s 439us/ste

584/584 [==============================] - 0s 454us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 390/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 391/500
584/584 [==============================] - 0s 444us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 392/500
584/584 [==============================] - 0s 444us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 393/500
584/584 [==============================] - 0s 439us/step - loss: 8.4543 - accuracy: 0.44860s - loss: 8.1458 - accuracy: 0.
Epoch 394/500
584/584 [==============================] - 0s 432us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 395/500
584/584 [==============================] - 0s 434us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 396/500
584/584 [==============================] - 0s 429us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 397/500
584/584 [==============================] - 0s 432us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 398/500
584/584 [==============

584/584 [==============================] - 0s 447us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 467/500
584/584 [==============================] - 0s 435us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 468/500
584/584 [==============================] - 0s 437us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 469/500
584/584 [==============================] - 0s 441us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 470/500
584/584 [==============================] - 0s 447us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 471/500
584/584 [==============================] - 0s 437us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 472/500
584/584 [==============================] - 0s 437us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 473/500
584/584 [==============================] - 0s 429us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 474/500
584/584 [==============================] - 0s 444us/step - loss: 8.4543 - accuracy: 0.4486
Epoch 475/500
584/584 [==============================] - 0s 435us/ste

In [70]:
#text = "We are looking for an experienced Audit professional in our Audit Department as “Assistant Manager - Operations” to be responsible for operational audits at the group level. Qualification: (CA / ACCA / CMA) Qualified / Finalist Experience: (Articles completed from Big four + 1 to 2 yrs of Operations audit experience) OR (5 yrs of Operational audit experience in FMCG or manufacturing concern) To: nasir.ahmed@dawn-group.com Mention Position in Subject Line Last date to apply: 18-04-2020"
#text = stemSentence(text)
#text = funct(text)
#text = vectorizer.fit_transform([text])
#classes = model.predict(text, batch_size=10)